In [140]:
import requests
import xmltodict, json
import pandas as pd
import numpy as np

In [7]:
key = 'xhgQoB2u7VqMuQGRKrDa6KLMfq2syfZeeAuEItSoVBfsL7hytyzbKBVbzAPD34JKyCtl3fbgQfdodniyjfueEw%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200315&endCreateDt=20210815'
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey={}'.format(key)

In [26]:
content = requests.get(url).content
dict1=xmltodict.parse(content)
jsonStr=json.dumps(dict1['response']['body']['items'], ensure_ascii=False)
jsonObj = json.loads(jsonStr)

df=pd.DataFrame(jsonObj['item'], index=['index','확진자수','확진률',])

# df.to_excel("누적확진자.xlsx")

In [28]:
cov=pd.read_excel("누적확진자.xlsx")

In [40]:
cov

,Unnamed: 0,confCase,confCaseRate,createDt,criticalRate,death,deathRate,gubun,seq,updateDt
0,0,11262,5.03,2021-08-15 16:46:14.429,0.00,0,0.00,0-9,10893,NaN
1,1,18494,8.26,2021-08-15 16:46:14.428,0.00,0,0.00,10-19,10892,NaN
2,2,39899,17.82,2021-08-15 16:46:14.428,0.02,6,0.28,20-29,10891,NaN
3,3,33113,14.79,2021-08-15 16:46:14.428,0.03,11,0.51,30-39,10890,NaN
4,4,35486,15.85,2021-08-15 16:46:14.428,0.05,19,0.88,40-49,10889,NaN
...,...,...,...,...,...,...,...,...,...,...
5459,5459,1257,12.60,2020-04-02 19:02:48.48,1.78,22,13.58,60-69,5,NaN
5460,5460,663,6.65,2020-04-02 19:02:48.48,7.07,46,28.40,70-79,4,NaN
5461,5461,453,4.54,2020-04-02 19:02:48.48,18.55,82,50.62,80 이상,3,NaN
5462,5462,5997,60.11,2020-04-02 19:02:48.48,1.36,80,49.38,여성,2,NaN


In [46]:
for i in range(len(cov)):
    cov.loc[i, 'createDt']=cov.loc[i, 'createDt'][:7]

In [194]:
cov_gu=list(set(cov['gubun']))
cov_gu.sort()
cov_gu
cov_Dt=list(set(cov['createDt']))
cov_Dt=sorted(cov_Dt)

In [209]:
cov_20_04 = []
for j in cov_Dt:
    cov4 = cov[cov['createDt'] == j]
    for i in cov_gu:
        cov4_0=cov4.copy()[cov4['gubun'] == i].reset_index()
        cov40=cov4_0.copy().loc[0]
        cov40['확진자'] =cov4_0.iloc[0]['confCase']-cov4_0.iloc[-1]['confCase']
        cov40['사망자'] = cov4_0.iloc[0]['death']-cov4_0.iloc[-1]['death']
        cov_20_04.append(cov40[2:])

df=pd.DataFrame(np.array(cov_20_04),columns=['confCase','confCaseRate','createDt','criticalRate','death','deathRate','gubun','seq','updateDt','월확진자','월사망자'])
df.to_excel("covid_19확진자(연령,성).xlsx")
df

,confCase,confCaseRate,createDt,criticalRate,death,deathRate,gubun,seq,updateDt,월확진자,월사망자
0,140,1.3,2020-04,0.0,0,0.0,0-9,453,NaN,21,0
1,590,5.48,2020-04,0.0,0,0.0,10-19,452,NaN,68,0
2,2952,27.42,2020-04,0.0,0,0.0,20-29,451,NaN,248,0
3,1158,10.76,2020-04,0.17,2,0.81,30-39,450,NaN,115,1
4,1427,13.26,2020-04,0.21,3,1.21,40-49,449,NaN,90,2
...,...,...,...,...,...,...,...,...,...,...,...
182,27725,12.38,2021-08,1.01,281,13.03,60-69,10887,NaN,1950,17
183,11677,5.21,2021-08,5.21,608,28.2,70-79,10886,NaN,606,15
184,6497,2.9,2021-08,17.47,1135,52.64,80 이상,10885,NaN,306,8
185,114536,51.15,2021-08,0.94,1074,49.81,남성,10883,NaN,12624,29
